## Phase 3: Environment Setup & Data Ingestion
This cell bootstraps the Colab environment by cloning the global repository, setting the working directory, and installing the strict version requirements defined in `requirements.txt`.

In [ ]:
import os

# 1. Clone the repository (if it doesn't exist in the Colab runtime yet)
if not os.path.exists('/content/paint-paint'):
    !git clone https://github.com/Binamra00/paint-paint.git

# 2. Change the working directory to the root of the cloned repo
%cd /content/paint-paint

# 3. Install strict dependencies
!pip install -q -r requirements.txt

print("Environment successfully bootstrapped!")

### Global Random Seeds & Data Download
To ensure this experiment is 100% reproducible, we lock all random number generators. Then, we execute our modular data ingestion script to fetch the Oxford 102 Flowers dataset.

In [ ]:
import torch
import numpy as np
import random

# Lock seeds for reproducibility
SEED = 42
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
np.random.seed(SEED)
random.seed(SEED)
torch.backends.cudnn.deterministic = True

# Execute our custom data ingestion pipeline module
!python data/download_oxford.py